In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
PATH_DATA = holo._PATH_DATA

FNAME_ILLUSTRIS_DATA = "illustris-galaxy-mergers_L75n1820FP_gas-100_dm-100_star-100_bh-000.hdf5"
FNAME_MCCONNELL_MA_2013 = "mcconnell+ma-2013_1211.2816.txt"

FNAME_DATA = os.path.join(PATH_DATA, FNAME_ILLUSTRIS_DATA)
FNAME_MCCONNELL_MA_2013 = os.path.join(PATH_DATA, FNAME_MCCONNELL_MA_2013)

In [ ]:
sepa = np.logspace(-4, 4, 100) * PC

fig, ax = utils.figax(xlabel='Separation [pc]', ylabel='Hardening Time [yr]')

data = evo.at('sepa', sepa)
dadt = np.fabs(data['dadt'])

conf = utils.quantiles(sepa/dadt/YR, sigmas=[0, -1, 1], axis=0)
med, *conf = conf.T

xx = sepa / PC

ax.plot(xx, med)
ax.fill_between(xx, *conf, alpha=0.2)

plt.show()

Compare actual evolution time to intended time (`htime`)

In [ ]:
time = evo.tlbk
dt = time[:, 0] - time[:, -1]

kale.dist1d(dt/htime, density=True)

# Quick: Population, Evolution, GW Spectrum

In [ ]:
# ---- Create initial population

pop = holo.Pop_Illustris(FNAME_DATA)

# ---- Apply population modifiers

# resample to increase the number of binaries
mod_resamp = holo.PM_Resample(resample=5.0)
# modify population (in-place)
pop.modify(mod_resamp)

# ---- Evolve binary population

# create a fixed-total-time hardening mechanism
fixed = holo.evolution.Fixed_Time(2.0 * GYR)
# add population and hardening to an evolution instance
evo = holo.evolution.Evolution(pop, fixed)
# evolve binary population
evo.evolve()

# ---- Calculate and Plot GWB

# construct sampling frequencies
freqs = holo.utils.nyquist_freqs(dur=20.0*YR, cad=0.3*YR)
# calculate discretized GW signals
gwb = holo.gravwaves.GW_Discrete(evo, freqs, nreals=30)
gwb.emit()

plot_gwb(gwb)
plt.show()

# Step-by-Step

## Construct Illustris-Based Binary Population

In [ ]:
pop = holo.Pop_Illustris(FNAME_DATA)
ill_name = os.path.basename(pop._fname).split('_')[1]
print("Loaded", pop.size, "binaries from Illustris", ill_name)

In [ ]:
plot.plot_bin_pop(pop)
plt.show()

### Apply a modifier to add (arbitrary) eccentricities, and resample 5x

In [ ]:
mod_eccen = holo.PM_Eccentricity()
mod_resamp = holo.PM_Resample(resample=2.5)

mods = [mod_eccen, mod_resamp]
pop.modify(mods)
print("Population now has", pop.size, "elements")
msg = np.percentile(pop.eccen, [25.0, 50.0, 75.0])
msg = ["{:.3f}".format(vv) for vv in msg]
msg = ", ".join(msg)
print("Eccentricities: ", msg)

In [ ]:
plot_bin_pop(pop)
plt.show()

### Apply Modifer to Use McConnell+Ma 2013 BH masses

In [ ]:
# Create the modifier using M-Mbulge relation
mod_mm13 = holo.PM_MM13(relation='mbulge')

# Choose percentiles
percs = 100*sp.stats.norm.cdf([-1, 0, 1])
percs = [0,] + percs.tolist() + [100,]

# Format nicely
str_array = lambda xx: ", ".join(["{:.2e}".format(yy) for yy in xx])
str_masses = lambda xx: str_array(np.percentile(xx/MSOL, percs))

# Modify population
print("Masses before: ", str_masses(pop.mass))
pop.modify(mod_mm13)
print("Masses after : ", str_masses(pop.mass))
    
plot_mbh_scaling_relations(pop, fname=FNAME_MCCONNELL_MA_2013)
plt.show()

# Binary Evolution

In [ ]:
bin_evo = holo.evolution.Evo_Magic_Delay_Eccen(pop)
bin_evo.evolve()

In [ ]:
freqs = np.logspace(-3, 1, 30) / YR
plot_evo(bin_evo, freqs)
plt.show()

# Calculate GWB

In [ ]:
freqs = holo.utils.nyquist_freqs(20.0, 0.3)
gwb = holo.GW_Discrete(bin_evo, freqs/YR, nreals=10)
gwb.emit()

In [ ]:
plot_gwb(gwb)
plt.show()